<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center> 

# ¿Cómo añadir más capas a continuación del transformer? 

En los notebooks anteriores, hemos trabajado con modelos transformers cuya arquitectura había sido extendida para tareas concretas, como por ejemplo la clasificación de textos. Dicha arquitectura ha sido encapsulada en una clase de HuggingFace, por ejemplo, **AutoModelForSequenceClassification**. 

En este notebook, aprenderemos a ajustar el modelo a una tarea, modificando u añadiendo nuevas capas específicas para la tarea sobre el modelo trasnformer. 
Además, también vamos a aprender cómo obtener los vectores del modelos transformer. 

Cuando tenemos muy pocos datos en el conjunto de entrenamiento, o si nuestra tarea es específica a un dominio, podríamos usar el conocimiento de otros modelos entrenados para otras tareas pero dentro del mismo dominio. Por ejemplo, supongamos que tenemos un conjunto de entrenamiento para la tarea de análisis de sentimiento en el dominio financiero, y que hay una gran cantidad de modelos que han sido entrenados para la tarea de QA en dicho dominio. La idea de es que podemos podemos aprovechar algunas capas de estos modelos para mejorar nuestra tarea de sentiment analysis. Otro ejemplo es por ejemplo si hemos entrenado un modelo multiclasificación de noticias falsas sobre el dataset Fakeddit (un dataset enorme). Dicho modelo podría ser utilizado para clasificar fake news en otro conjunto de datos cuando queremos aprender a clasificar (binaria) noticias falsas entrenadas en el conjunto de datos Fakkedit (un conjunto de datos enorme). Lo que haríamos sería aprovechar las capas de la tarea de multiclasificación, agregando nuestra propia capa para la clasificación binaria. 


## Transfer learning de analisis de sentimiento a detección de sarcamos. 
 
 A continuación, vamos a utilizar un modelo que fue entrenado para clasificar tweets en 5 emociones. Vamos a añadir capas adicionales para dirigir una nueva tarea, la detección de sarcamos, que es un problema de clasificación binaria. 

In [ ]:
! pip install transformers datasets evaluate

Cargamos el modelo entrenado para la tarea de análisis de sentimiento. 


In [ ]:
from transformers import AutoTokenizer
model_name = "cardiffnlp/twitter-roberta-base-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 'RobertaTokenizerFast' no tiene el atributo 'model_max_len', así que debemos definirlo
tokenizer.model_max_len=512

Cargamos ahora el dataset para la tarea de detección de sarcasmos: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
PATH_DATA = '/content/drive/My Drive/Colab Notebooks/data/sarcasm/'


In [ ]:
from datasets import DatasetDict, load_dataset
from transformers import DataCollatorWithPadding

dataset_dict = load_dataset("csv", data_files={'train':PATH_DATA+"train.csv", "val":PATH_DATA+"val.csv", "test":PATH_DATA+"test.csv"})
dataset_dict

dataset_dict = dataset_dict.remove_columns(['Unnamed: 0'])
dataset_dict = dataset_dict.rename_columns({'headline':'text','is_sarcastic':'label'})

# now, we encoding the data
def tokenize(batch):
  return tokenizer(batch["text"], truncation=True, max_length=512)

encoded_dataset = dataset_dict.map(tokenize, batched=True)

# creamos un datacollator que pasará los datos al modelo 
encoded_dataset.set_format("torch",columns=["input_ids", "attention_mask", "label"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
encoded_dataset

Ahora, vamos a crear e nuestro propio modelo. Primero vamos a extraer la cabeza del modelo entrenado para la tarea de análisis de sentimento. Luego vamos a añadir una capa adicional para la tarea de clasificación binaria en la detección de sarcasmo. 

Creamos una clase, **CustomModel**, cuyo constructor recibe como parámetro el nombre del modelo y el número de labels. El constructor va a extraer el cuerpo de dicho modelo usando la clase **AutoModel**.



Luego, usamos **TokenClassifierOutput** para asegurarnos de que la salida sea similar a la salida del modelo original.

In [ ]:
from transformers import AutoModel, AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn       #layes for NN

class CustomModel(nn.Module):

  def __init__(self,checkpoint,num_labels): 

    super(CustomModel,self).__init__() 
    self.num_labels = num_labels 

    self.model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    self.dropout = nn.Dropout(0.1) 
    self.classifier = nn.Linear(768,num_labels) 

  def forward(self, input_ids=None, attention_mask=None,labels=None):
    #utiliza el modelo para generar la salida
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

    #aplica el resto de capas
    sequence_output = self.dropout(outputs[0]) #outputs[0]=último estado
    logits = self.classifier(sequence_output[:,0,:].view(-1,768)) # calcula el error
    
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    
    return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states,attentions=outputs.attentions)

Vamos a entrenar el modelo:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=CustomModel(checkpoint= model_name, num_labels=2).to(device)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    encoded_dataset["train"], shuffle=True, batch_size=32, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    encoded_dataset["val"], batch_size=32, collate_fn=data_collator
)

In [ ]:
from transformers import AdamW,get_scheduler
import evaluate 
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

metric = evaluate.load("f1")

In [ ]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs * len(eval_dataloader)))


for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar_train.update(1)

  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar_eval.update(1)
    
  print(metric.compute())

Evaluamos ahora sobre el conjunto test:

In [ ]:
model.eval()

test_dataloader = DataLoader(
    encoded_dataset["test"], batch_size=32, collate_fn=data_collator
)

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()